# SparkSQL Partitioning Solution

First of all load the flights data from a JSON file into a DataFrame. Then we will select the columns that we will be use for the flight edge DataFrame. The required column names are id, src, and dst, so we rename those columns in the select statement

In [1]:
flightdata = spark.read.option("inferSchema", "false").json("data/flightsdata.json");

flightdata.printSchema();

root
 |-- _id: string (nullable = true)
 |-- arrdelay: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- crsarrtime: long (nullable = true)
 |-- crsdephour: long (nullable = true)
 |-- crsdeptime: long (nullable = true)
 |-- crselapsedtime: double (nullable = true)
 |-- depdelay: double (nullable = true)
 |-- dest: string (nullable = true)
 |-- dist: double (nullable = true)
 |-- dofW: long (nullable = true)
 |-- origin: string (nullable = true)



In [2]:
flights = flightdata.withColumnRenamed("_id", "id").withColumnRenamed("origin", "src").withColumnRenamed("dest", "dst").withColumnRenamed("depdelay", "delay");

flights.printSchema();
flights.show();

root
 |-- id: string (nullable = true)
 |-- arrdelay: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- crsarrtime: long (nullable = true)
 |-- crsdephour: long (nullable = true)
 |-- crsdeptime: long (nullable = true)
 |-- crselapsedtime: double (nullable = true)
 |-- delay: double (nullable = true)
 |-- dst: string (nullable = true)
 |-- dist: double (nullable = true)
 |-- dofW: long (nullable = true)
 |-- src: string (nullable = true)

+--------------------+--------+-------+----------+----------+----------+--------------+-----+---+------+----+---+
|                  id|arrdelay|carrier|crsarrtime|crsdephour|crsdeptime|crselapsedtime|delay|dst|  dist|dofW|src|
+--------------------+--------+-------+----------+----------+----------+--------------+-----+---+------+----+---+
|AA_2017-01-01_ATL...|     0.0|     AA|      1912|        17|      1700|         132.0|  0.0|LGA| 762.0|   7|ATL|
|AA_2017-01-01_LGA...|     0.0|     AA|      1620|        13|      1343|         1

## Parquet Partitioning
Spark table's partitioning optimizes reads by storing files into hierarchy of directories based on columns using which they are partitioned.
Here is the code to persist a flights DataFrame as a table consisting of Parquet files partitioned by the src column:

In [3]:
flights.write.mode("overwrite").format("parquet").partitionBy("src").option("path", "data/flights").saveAsTable("flights")

Below is the resulting directory structure as shown by a Hadoop list files command:

In [4]:
!hdfs dfs -ls data/flights

Found 10 items
-rw-r--r--   1 training supergroup          0 2021-03-08 17:17 data/flights/_SUCCESS
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=ATL
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=BOS
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=DEN
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=EWR
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=IAH
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=LGA
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=MIA
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=ORD
drwxr-xr-x   - training supergroup          0 2021-03-08 17:17 data/flights/src=SFO


Below, we see that the src=ATL subdirectory contains Parquet files:

In [5]:
!hdfs dfs -ls data/flights/src=ATL

Found 2 items
-rw-r--r--   1 training supergroup      59191 2021-03-08 17:17 data/flights/src=ATL/part-00000-5910ff2c-0ef5-4710-bd1b-24e7442910b5.c000.snappy.parquet
-rw-r--r--   1 training supergroup      25909 2021-03-08 17:17 data/flights/src=ATL/part-00001-5910ff2c-0ef5-4710-bd1b-24e7442910b5.c000.snappy.parquet


## Partition Pruning and Predicate Pushdown
**Partition pruning** 
>It is a performance optimization that limits the number of files and partitions that Spark reads when querying.<br> 
>After partitioning, queries which certainly matches the filter criteria partitions for improving the performance by allowing Spark to only read a subset of the directories and files.<br> 
>When partition filters are present, the catalyst optimizer pushes down the partition filters.<br> 
>The scan reads only the directories that match the partition filters, which actually reduces the disk I/O.<br> 
>For example, the following query reads only the files in the src=DEN partition directory in order to query the average departure delay for flights originating from Denver.

In [6]:
from pyspark.sql.functions import desc
flights.filter("src = 'DEN' and delay > 1").groupBy("src", "dst").avg("delay").sort(desc("avg(delay)")).show()

+---+---+------------------+
|src|dst|        avg(delay)|
+---+---+------------------+
|DEN|SFO| 51.55853658536585|
|DEN|EWR|47.881147540983605|
|DEN|LGA| 41.92549019607843|
|DEN|ORD| 40.98522167487685|
|DEN|ATL| 36.64888888888889|
|DEN|IAH|28.633333333333333|
|DEN|BOS|24.432989690721648|
|DEN|MIA|22.155172413793103|
+---+---+------------------+



One can see the physical plan for a DataFrame query in the Spark web UI SQL tab or by calling the explain method. 

In [7]:
flights.filter("src = 'DEN' and delay > 1").groupBy("src", "dst").avg("delay").sort(desc("avg(delay)")).explain(True)

== Parsed Logical Plan ==
'Sort ['avg(delay) DESC NULLS LAST], true
+- Aggregate [src#44, dst#57], [src#44, dst#57, avg(delay#70) AS avg(delay)#234]
   +- Filter ((src#44 = DEN) AND (delay#70 > cast(1 as double)))
      +- Project [id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14 AS delay#70, dst#57, dist#16, dofW#17L, src#44]
         +- Project [id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14, dest#15 AS dst#57, dist#16, dofW#17L, src#44]
            +- Project [id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14, dest#15, dist#16, dofW#17L, origin#18 AS src#44]
               +- Project [_id#7 AS id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14, dest#15, dist#16, dofW#17L, origin#18]
                  +- RelationV2[_id#7, arrdelay#8, carrier#9, crsa

The physical plan is read from the bottom up, whereas the DAG is read from the top down. Note: the Exchange means a shuffle occurred between stages.

<img src="https://i.ibb.co/QNKMdCJ/1.jpg" alt="1" border="0">

## Partitioning Tips
The partition columns must be used frequently in queries for filtering and must have a small range of values with corresponding data for distributing the files in the directories. 
If you want to avoid too many small files, which actually make scans less efficient with parallelism. You also should avoid having too few large files, which can hurt parallelism.

## Coalesce and Repartition

Before or when writing a DataFrame, you can use dataframe.coalesce(N) to reduce the number of partitions in a DataFrame, without shuffling, or df.repartition(N) to reorder and either increase or decrease the number of partitions with shuffling data across the network to achieve even load balancing.


In [8]:
flights.repartition(13).write.mode("overwrite").format("parquet").partitionBy("src").option("path", "data/flights").saveAsTable("flights");

## Bucketing
* Bucketing is one of the data organization technique which groups data with the same bucket value within a fixed number of “buckets.” 
* It can improve performance in wide transformations and joins by avoiding “shuffles.” 
* Bucketing is similar to partitioning, but in partitions we generally creates a directory for each partitions, whereas in bucketing data is distributed across a fixed number of buckets by a hash on the bucket value.

In [9]:
flights.write.format("parquet").sortBy("id").partitionBy("src").bucketBy(4,"dst","carrier").option("path","data/flightsbkdc").saveAsTable("flightsbkdc")

The resulting directory structure is the same as before, with the files in the src directories bucketed by dst and carrier. The code below computes statistics on the table, which can then be used by the Catalyst optimizer. After that, the partitioned and bucketed table is read into a new DataFrame df2.

In [10]:
spark.sql("ANALYZE TABLE flightsbkdc COMPUTE STATISTICS")
df2  = spark.table("flightsbkdc")

Next, let’s look at the optimizations for the following query:

In [11]:
df2.filter("src = 'DEN' and delay > 1").groupBy("src", "dst","carrier").avg("delay").sort(desc("avg(delay)")).show()

+---+---+-------+------------------+
|src|dst|carrier|        avg(delay)|
+---+---+-------+------------------+
|DEN|LGA|     DL| 82.03333333333333|
|DEN|SFO|     UA|57.437275985663085|
|DEN|EWR|     UA|57.027027027027025|
|DEN|ATL|     DL| 42.94736842105263|
|DEN|ORD|     UA| 41.07801418439716|
|DEN|ORD|     AA|40.774193548387096|
|DEN|SFO|     WN|39.038167938931295|
|DEN|EWR|     WN|          33.78125|
|DEN|LGA|     UA| 32.66386554621849|
|DEN|MIA|     UA|              31.0|
|DEN|IAH|     UA|28.633333333333333|
|DEN|ATL|     WN|  28.0561797752809|
|DEN|BOS|     UA|26.966101694915253|
|DEN|LGA|     WN|24.763157894736842|
|DEN|BOS|     WN|              20.5|
|DEN|MIA|     AA| 16.34285714285714|
|DEN|ATL|     UA|12.333333333333334|
+---+---+-------+------------------+



Here, we can see partition filter and filter pushdown, but we can also see that there is no “Exchange” like before bucketing, which means there was no shuffle to aggregate by src, dst, and carrier.

<img src="https://i.ibb.co/fH3hfSd/2.jpg" alt="2" border="0">

In the DAG(Direct Acyclic Graph), we can see that there is no exchange shuffle, and we can also see “Whole-Stage Java Code Generation,” which actually optimizes CPU usage, by generating a single optimized function in bytecode.

<img src="https://i.ibb.co/GPzXNwC/3.jpg" alt="3" border="0">

## Projection and Filter Pushdown
* Below, we see the physical plan for a DataFrame query, with projection and filter pushdown. 
* This means that the scanning of the src, dst, and depdelay columns and the filter on the depdelay column are pushed down into Source.
* Meaning that the scanning and filtering will take place in Source before than returning the data to Spark. 
* Projection pushdown actually minimizes the data transfer between the Source and the Spark engine by removing the unnecessary fields from table scans. 
* It is especially beneficial when a table contains many columns. 
* Filter pushdown actually improves performance by reducing the amount of the data passed between the Source and the Spark engine when filtering data.


In [12]:
flights.filter("src = 'ATL' and delay > 1").groupBy("src", "dst").avg("delay").sort(desc("avg(delay)")).explain(True)

== Parsed Logical Plan ==
'Sort ['avg(delay) DESC NULLS LAST], true
+- Aggregate [src#44, dst#57], [src#44, dst#57, avg(delay#70) AS avg(delay)#440]
   +- Filter ((src#44 = ATL) AND (delay#70 > cast(1 as double)))
      +- Project [id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14 AS delay#70, dst#57, dist#16, dofW#17L, src#44]
         +- Project [id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14, dest#15 AS dst#57, dist#16, dofW#17L, src#44]
            +- Project [id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14, dest#15, dist#16, dofW#17L, origin#18 AS src#44]
               +- Project [_id#7 AS id#31, arrdelay#8, carrier#9, crsarrtime#10L, crsdephour#11L, crsdeptime#12L, crselapsedtime#13, depdelay#14, dest#15, dist#16, dofW#17L, origin#18]
                  +- RelationV2[_id#7, arrdelay#8, carrier#9, crsa

## End of Exercise